# Advent of Code 2024

## Day 0: Imports and Utility Functions

In [1]:
import math
import numpy as np
import re
from collections import Counter, deque, namedtuple
from functools import cmp_to_key
from itertools import product, combinations
from typing import List

In [2]:
def file_to_list(filename, sep="\n", maxsplit=-1) -> List[str]:
    """
    Read an input file and split it using sep as the delimiter.
    """
    with open(filename) as f:
        return f.read().rstrip().split(sep, maxsplit=maxsplit)

## Day 1: Historian Hysteria

### Part 1

In [3]:
def day1part1():
    input_lines = file_to_list("./inputs/input1.txt")
    list_1 = []
    list_2 = []
    for line in input_lines:
        val1, val2 = line.split("   ")
        list_1.append(int(val1))
        list_2.append(int(val2))
    list_1.sort()
    list_2.sort()
    distance = 0
    for x, y in zip(list_1, list_2):
        distance += abs(x - y)
    return(distance)

day1part1()

2904518

### Part 2

In [4]:
def day1part2():
    input_lines = file_to_list("./inputs/input1.txt")
    list_1 = []
    list_2 = []
    for line in input_lines:
        val1, val2 = line.split("   ")
        list_1.append(int(val1))
        list_2.append(int(val2))
    counts = Counter(list_2)
    sim_score = sum([(val * counts.get(val, 0)) for val in list_1])
    return(sim_score)

day1part2()

18650129

## Day 2: Red-Nosed Reports

### Part 1

In [5]:
test_reports = [
    "7 6 4 2 1",
    "1 2 7 8 9",
    "9 7 6 2 1",
    "1 3 2 4 5",
    "8 6 4 4 1",
    "1 3 6 7 9"
]

In [6]:
def day2part1(reports):
    reports = [list(map(int, report.split(" "))) for report in reports]
    safe_count = 0
    for report in reports:
        if ((sorted(report, reverse=True) == report) or
            (sorted(report, reverse=False) == report)):
            diffs = [report[i+1] - report[i] for i in range(len(report)-1)]
            if (all(diff in {-3, -2, -1, 1, 2, 3} for diff in diffs)):
                safe_count += 1
    return(safe_count)

day2part1(test_reports)

2

In [7]:
final_reports = file_to_list("./inputs/input2.txt")
day2part1(final_reports)

663

### Part 2

In [8]:
def is_safe(report):
    if ((sorted(report, reverse=True) == report) or
        (sorted(report, reverse=False) == report)):
            diffs = [report[i+1] - report[i] for i in range(len(report)-1)]
            if (all(diff in {-3, -2, -1, 1, 2, 3} for diff in diffs)):
                return True
    return False


def day2part2(reports):
    reports = [list(map(int, report.split(" "))) for report in reports]
    safe_count = 0
    for report in reports:
        report_is_safe = False
        for i in range(len(report)):
            report_mutated = report[:i] + report[i+1 :]
            if is_safe(report_mutated):
                report_is_safe = True
        if report_is_safe:
             safe_count += 1
    return(safe_count)

day2part2(test_reports)

4

In [9]:
day2part2(final_reports)

692

## Day 3: Mull It Over

### Part 1

In [10]:
test_memory = "xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))"

In [11]:
def day3_part1(memory):
    matches = re.findall("mul\([0-9]+,[0-9]+\)", memory)
    sum = 0
    for match in matches:
        nums = list(map(int, match[4:-1].split(",")))
        sum += nums[0] * nums[1]
    return sum

day3_part1(test_memory)

161

In [12]:
real_memory = open("./inputs/input3.txt").read()
day3_part1(real_memory)

189527826

### Part 2

In [13]:
test_memory = "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"

In [14]:
def day3_part1(memory):
    pattern = "(mul\([0-9]+,[0-9]+\))|(do\(\))|(don't\(\))"
    matches = re.findall(pattern, memory)
    sum = 0
    do = True
    for match in matches:
        if match[1] == "do()":
            do = True
        elif match[2] == "don't()":
            do = False
        else:
            if do:
                nums = list(map(int, match[0][4:-1].split(",")))
                sum += nums[0] * nums[1]
    return sum

day3_part1(test_memory)

48

In [15]:
day3_part1(real_memory)

63013756

## Day 4: Ceres Search

### Part 1

In [16]:
example_word_search = [
    "MMMSXXMASM",
    "MSAMXMSMSA",
    "AMXSXMAAMM",
    "MSAMASMSMX",
    "XMASAMXAMM",
    "XXAMMXXAMA",
    "SMSMSASXSS",
    "SAXAMASAAA",
    "MAMMMXMMMM",
    "MXMXAXMASX"
]

In [17]:
def day4_part1(word_search):
    word_search = [[char for char in line] for line in word_search]
    word_search = np.array(word_search)
    count = 0
    for word_search in [word_search.copy(), np.flipud(word_search), np.fliplr(word_search)]:
        for _ in range(4):
            for line in word_search:
                line = "".join(line.tolist())
                count += len(re.findall("XMAS", line))
            word_search = np.rot90(word_search)
    return count

day4_part1(example_word_search)

24

In [18]:
def day4_part1(grid):
    rows = len(grid)
    cols = len(grid[0])
    word = "XMAS"
    word_length = len(word)
    
    directions = [
        (0, 1),   # right
        (0, -1),  # left
        (1, 0),   # down
        (-1, 0),  # up
        (1, 1),   # down-right
        (1, -1),  # down-left
        (-1, 1),  # up-right
        (-1, -1)  # up-left
    ]
    count = 0
    for r in range(rows):
        for c in range(cols):
            # Only proceed if the first letter matches
            if grid[r][c] == word[0]:
                # Check each direction
                for dr, dc in directions:
                    # Check if we can find the whole word in this direction
                    found = True
                    for i in range(word_length):
                        rr = r + dr * i
                        cc = c + dc * i
                        # Check boundaries and character match
                        if not (0 <= rr < rows and 0 <= cc < cols and grid[rr][cc] == word[i]):
                            found = False
                            break
                    if found:
                        count += 1
    return count

day4_part1(example_word_search)

18

In [19]:
real_word_search = file_to_list("./inputs/input4.txt")
day4_part1(real_word_search)

2297

### Part 2

In [20]:
def day4_part2(grid):
    # Patterns for one diagonal
    mas = ['M', 'A', 'S']
    sam = ['S', 'A', 'M']
    # All combinations of diagonals
    diagonal_patterns = [
        (mas, mas),
        (mas, sam),
        (sam, mas),
        (sam, sam)
    ]
    
    rows = len(grid)
    cols = len(grid[0])
    
    count = 0
    
    # Iterate over every starting point of a 3x3 sub-grid
    for r in range(rows - 2):  # up to rows-3
        for c in range(cols - 2):  # up to cols-3
            # Extract diagonals
            # TL-BR diagonal
            diag1 = [grid[r][c], grid[r+1][c+1], grid[r+2][c+2]]
            # TR-BL diagonal
            diag2 = [grid[r][c+2], grid[r+1][c+1], grid[r+2][c]]
            
            # Check if (diag1, diag2) matches any allowed pattern
            for d1, d2 in diagonal_patterns:
                if diag1 == d1 and diag2 == d2:
                    count += 1
    
    return count

day4_part2(example_word_search)

9

In [21]:
day4_part2(real_word_search)

1745

## Day 5: Print Queue

### Part 1

In [22]:
test_rules = [
    "47|53",
    "97|13",
    "97|61",
    "97|47",
    "75|29",
    "61|13",
    "75|53",
    "29|13",
    "97|29",
    "53|29",
    "61|53",
    "97|53",
    "61|29",
    "47|13",
    "75|47",
    "97|75",
    "47|61",
    "75|61",
    "47|29",
    "75|13",
    "53|13"
]
test_updates = [
    "75,47,61,53,29",
    "97,61,53,29,13",
    "75,29,13",
    "75,97,47,61,53",
    "61,13,29",
    "97,13,75,29,47"
]

In [23]:
def day5_part1(rules, updates):
    correct_updates = []
    for update in updates:
        update = update.split(",")
        correct = True
        for rule in rules:
            left, right = rule.split("|")
            if (left in update) and (right in update):
                idx_left = update.index(left)
                idx_right = update.index(right)
                if idx_left > idx_right:
                    correct = False
        if correct == True:
            correct_updates.append(update)
    sum = 0
    for update in correct_updates:
        middle_number = int(update[len(update) //  2])
        sum += middle_number
    return sum

day5_part1(test_rules, test_updates)

143

In [24]:
final_rules, final_updates = [block.split("\n") for block in file_to_list("./inputs/input5.txt", sep = "\n\n")]
day5_part1(final_rules, final_updates)

6260

### Part 2

In [25]:
def day5_part2(rules, updates):
    incorrect_updates = []
    for update in updates:
        update = update.split(",")
        correct = True
        for rule in rules:
            left, right = rule.split("|")
            if (left in update) and (right in update):
                idx_left = update.index(left)
                idx_right = update.index(right)
                if idx_left > idx_right:
                    correct = False
        if correct == False:
            incorrect_updates.append(update)
    cmp = cmp_to_key(lambda x,y: 1-2*(x+'|'+y in rules))
    sum = 0
    for update in incorrect_updates:
        update = sorted(update, key = cmp)
        middle_number = int(update[len(update) //  2])
        sum += middle_number
    return sum

day5_part2(test_rules, test_updates)

123

In [26]:
day5_part2(final_rules, final_updates)

5346

## Day 6: Guard Gallivant

### Part 1

In [27]:
test_chart_input = [
    "....#.....",
    ".........#",
    "..........",
    "..#.......",
    ".......#..",
    "..........",
    ".#..^.....",
    "........#.",
    "#.........",
    "......#..."
]

In [28]:
class Chart:

    def __init__(self, chart):
        self.chart = np.array([list(row) for row in chart])

    def get_position_guard(self):
        position = None
        if np.any(np.isin(self.chart, ">")):
            position = np.where(self.chart == ">")
        elif np.any(np.isin(self.chart, "<")):
            position = np.where(self.chart == "<")
        elif np.any(np.isin(self.chart, "^")):
            position = np.where(self.chart == "^")
        elif np.any(np.isin(self.chart, "v")):
            position = np.where(self.chart == "v")
        position = (position[0][0], position[1][0])
        return(position)
    
    def advance_guard(self):
        position = self.get_position_guard()
        direction = self.chart[position]
        if direction == ">":
            new_position = (position[0], position[1] + 1)
        elif direction == "<":
            new_position = (position[0], position[1] - 1)
        elif direction == "^":
            new_position = (position[0] - 1, position[1])
        elif direction == "v":
            new_position = (position[0] + 1, position[1])
        if (new_position[0] > len(self.chart) - 1) or (new_position[1] > len(self.chart[1]) - 1):
            return(False)
        if self.chart[new_position] == "#":
            new_direction = {"^": ">", ">": "v", "v": "<", "<": "^"}.get(direction)
            new_position = position
        else:
            new_direction = direction
        self.chart[position] = "."
        self.chart[new_position] = new_direction
        return(True)


def day6_part1(chart: Chart):
    visited = set()
    visited.add(chart.get_position_guard())
    while chart.advance_guard():
        current_position = chart.get_position_guard()
        if current_position not in visited:
            visited.add(current_position)
    return len(visited)

test_chart = Chart(test_chart_input)
day6_part1(test_chart)

41

In [29]:
final_chart = Chart(file_to_list("./inputs/input6.txt"))
day6_part1(final_chart)

4977

## Day 7: Bridge Repair

### Part 1

In [30]:
test_equations = [
    "190: 10 19",
    "3267: 81 40 27",
    "83: 17 5",
    "156: 15 6",
    "7290: 6 8 6 15",
    "161011: 16 10 13",
    "192: 17 8 14",
    "21037: 9 7 18 13",
    "292: 11 6 16 20"
]

In [31]:
def day7_part1(equations):
    sum = 0
    for equation in equations:
        equation = equation.split(": ")
        value = int(equation[0])
        nums = [int(num) for num in equation[1].split(" ")]
        for ops in product("+*", repeat=len(nums)-1):
            result = nums[0]
            for idx, op in enumerate(ops):
                match op:
                    case "*": result = result * nums[idx+1]
                    case "+": result = result + nums[idx+1]
            if result == value:
                sum += value
                break
    return(sum)

day7_part1(test_equations)

3749

In [32]:
final_equations = file_to_list("./inputs/input7.txt")
day7_part1(final_equations)

465126289353

### Part 2

In [33]:
def day7_part2(equations):
    sum = 0
    for equation in equations:
        equation = equation.split(": ")
        value = int(equation[0])
        nums = [int(num) for num in equation[1].split(" ")]
        for ops in product("+*|", repeat=len(nums)-1):
            result = nums[0]
            for idx, op in enumerate(ops):
                match op:
                    case "*": result = result * nums[idx+1]
                    case "+": result = result + nums[idx+1]
                    case "|": result = int(str(result) + str(nums[idx+1]))
            if result == value:
                sum += value
                break
    return(sum)

day7_part2(test_equations)

11387

In [34]:
day7_part2(final_equations)

70597497486371

## Day 8: Resonant Collinearity

### Part 1

In [35]:
example_chart = [
    "............",
    "........0...",
    ".....0......",
    ".......0....",
    "....0.......",
    "......A.....",
    "............",
    "............",
    "........A...",
    ".........A..",
    "............",
    "............"
]

In [36]:
def day8_part1(chart):
    antenas = {}
    N = len(chart)
    M = len(chart[0])
    for i in range(N):
        for j in range(M):
            freq = chart[i][j]
            if freq != ".":
                l = antenas.get(freq, [])
                l.append((i, j))
                antenas[freq] = l
    antinodes = set()
    for freq in antenas.keys():
        coords = antenas[freq]
        for pair in combinations(coords, 2):
            dist_n = pair[0][0] - pair[1][0]
            dist_m = pair[0][1] - pair[1][1]
            beacon_1 = (pair[0][0] + dist_n, pair[0][1] + dist_m)
            beacon_2 = (pair[1][0] - dist_n, pair[1][1] - dist_m)
            if (0 <= beacon_1[0] < N) and (0 <= beacon_1[1] < M):
                # x, y = beacon_1
                # chart[x] = chart[x][:y] + "#" + chart[x][y+1:]
                antinodes.add(beacon_1)
            if (0 <= beacon_2[0] < N) and (0 <= beacon_2[1] < M):
                # x, y = beacon_2
                # chart[x] = chart[x][:y] + "#" + chart[x][y+1:]
                antinodes.add(beacon_2)
    # for row in chart:
    #     print(row)
    return len(antinodes)

day8_part1(example_chart.copy())

14

In [37]:
final_chart = file_to_list("./inputs/input8.txt")
day8_part1(final_chart.copy())

273

### Part 2

In [38]:
def day8_part2(chart):
    antenas = {}
    N = len(chart)
    M = len(chart[0])
    for i in range(N):
        for j in range(M):
            freq = chart[i][j]
            if freq != ".":
                l = antenas.get(freq, [])
                l.append((i, j))
                antenas[freq] = l
    antinodes = set()
    for freq in antenas.keys():
        coords = antenas[freq]
        for pair in combinations(coords, 2):
            dist_n = pair[0][0] - pair[1][0]
            dist_m = pair[0][1] - pair[1][1]
            # iterate a over a large enoough number to make sure we cover all possible antinodes
            for i in range(50):
                beacon_1 = (pair[0][0] + dist_n * i, pair[0][1] + dist_m * i)
                beacon_2 = (pair[1][0] - dist_n * i, pair[1][1] - dist_m * i)
                if (0 <= beacon_1[0] < N) and (0 <= beacon_1[1] < M):
                    antinodes.add(beacon_1)
                if (0 <= beacon_2[0] < N) and (0 <= beacon_2[1] < M):
                    antinodes.add(beacon_2)
    return len(antinodes)

day8_part2(example_chart.copy())

34

In [39]:
day8_part2(final_chart.copy())

1017

## Day 9: Disk Fragmenter

### Part 1

In [40]:
example_disk_map = "2333133121414131402"
example_disk_map = "233313312141413140211"

In [41]:
def day9_part1(disk_map):
    # Represent the disk layout
    blocks = []
    id = 0
    free_space = True
    for digit in disk_map:
        if free_space is True:
            blocks += [str(id)] * int(digit)
            id += 1
        else:
            blocks += ["."] * int(digit)
        free_space = True if free_space is False else False
    # Compress the disk
    while True:
        try:
            idx = blocks.index(".")
            blocks[idx] = blocks[-1]
            blocks.pop()
        except ValueError:
            break
    # Compute checksum
    checksum = 0
    for idx, number in enumerate(blocks):
        checksum += int(idx) * int(number)
    return checksum

day9_part1(example_disk_map)

2132

In [42]:
final_disk_map = file_to_list("./inputs/input9.txt")[0]
day9_part1(final_disk_map)

6366665108136

### Part 2

In [43]:
def day9_part2(disk_map):
    # Represent the disk layout
    blocks = []
    id = 0
    free_space = True
    for digit in disk_map:
        if free_space is True:
            if digit != "0":
                blocks += [str(id) * int(digit)]
            id += 1
        else:
            if digit != "0":
                blocks += ["." * int(digit)]
        free_space = True if free_space is False else False
    # Compress the disk
    for i, block in reversed([(idx, block) for idx, block in enumerate(blocks.copy()) if "." not in block]):
        for j, space in enumerate(blocks):
            if "." in space and len(space) >= len(block) and j < i:
                i = blocks.index(block)
                blocks[j] = block
                blocks.pop(i)
                blocks.insert(i, "." * len(block))
                if len(space) > len(block):
                    blocks.insert(j+1, "." * (len(space) - len(block)))
                break
        for k, space in enumerate(blocks):
            if k < len(blocks) - 1:
                if "." in space and "." in blocks[k+1]:
                    blocks[k] = blocks[k] + blocks.pop(k+1)
    print(blocks)
    # print("".join(blocks))
    # Compute checksum
    checksum = 0
    for idx, block in enumerate("".join(blocks)):
        if "." not in block:
            for digit in block:
                checksum += idx * int(digit)
    return checksum

# Works
day9_part2(example_disk_map)

['00', '10', '2', '111', '99', '..', '777', '333', '.', '44', '.', '5555', '.', '6666', '.....', '8888', '.....']


3108

In [44]:
# Doesn't work
# day9_part2(final_disk_map)

## Day 10: Hoof It

### Part 1

In [45]:
example_map = [
    "89010123",
    "78121874",
    "87430965",
    "96549874",
    "45678903",
    "32019012",
    "01329801",
    "10456732",
]

In [46]:
def day10_part1(input_map):
    Node = namedtuple("Node", ["i", "j"])
    trailheads = []
    for i in range(len(input_map)):
        for j in range(len(input_map[0])):
            if input_map[i][j] == "0":
                trailheads.append(Node(i, j))
    result = 0
    for trailhead in trailheads:
        score = 0
        # Breadth-first search
        queue = deque([trailhead])
        explored = set([trailhead])
        while queue:
            node = queue.popleft()
            if input_map[node.i][node.j] == "9":
                score += 1
            adjacent_nodes = [
                Node(node.i + 1, node.j),
                Node(node.i - 1, node.j),
                Node(node.i, node.j + 1),
                Node(node.i, node.j - 1)
            ]
            for adjacent in adjacent_nodes:
                if (0 <= adjacent.i < len(input_map)) and (0 <= adjacent.j < len(input_map[0])):
                    if adjacent not in explored:
                        current_value = int(input_map[node.i][node.j])
                        adjacent_value = int(input_map[adjacent.i][adjacent.j])
                        if adjacent_value == (current_value + 1):
                            explored.add(adjacent)
                            queue.append(adjacent)
        result += score
    return result

day10_part1(example_map)

36

In [47]:
final_map = file_to_list("./inputs/input10.txt")
day10_part1(final_map)

717

### Part 2

In [48]:
def day10_part2(input_map):
    Node = namedtuple("Node", ["i", "j"])
    trailheads = []
    for i in range(len(input_map)):
        for j in range(len(input_map[0])):
            if input_map[i][j] == "0":
                trailheads.append(Node(i, j))
    result = 0
    for trailhead in trailheads:
        paths = []
        # Breadth-first search
        queue = deque([[trailhead]])
        while queue:
            path = queue.popleft()
            node = path[-1]
            if input_map[node.i][node.j] == "9":
                paths.append(path)
            adjacent_nodes = [
                Node(node.i + 1, node.j),
                Node(node.i - 1, node.j),
                Node(node.i, node.j + 1),
                Node(node.i, node.j - 1)
            ]
            for adjacent in adjacent_nodes:
                if (0 <= adjacent.i < len(input_map)) and (0 <= adjacent.j < len(input_map[0])):
                    current_value = int(input_map[node.i][node.j])
                    adjacent_value = int(input_map[adjacent.i][adjacent.j])
                    if adjacent_value == (current_value + 1):
                        new_path = list(path)
                        new_path.append(adjacent)
                        queue.append(new_path)
        result += len(paths)
    return result

day10_part2(example_map)

81

In [49]:
day10_part2(final_map)

1686